# Capa adicional de clasificacion de finger movements

Realizamos una capa adicional de clasificación sobre las formas tokenizadas generadas por VQShape. Aunque VQShape se entrena principalmente como un extractor de características — proporcionando una representación en forma de tokens y un histograma de códigos para series de tiempo — no incluye de por sí una cabeza de clasificación supervisada.

Aprovechamos ese modelo pre-entrenado como feature extractor, y sobre su salida aplicamos un clasificador lineal que permite distinguir los movimientos de dedos. El entrenamiento fue realizado en un entorno de Google Colab, aprovechando una GPU para acelerar el proceso.

Como estamos en un entorno colab clonamos el repositorio original y nos vamos a el directorio VQShape

In [1]:
!git clone https://github.com/YunshiWen/VQShape.git
%cd VQShape


fatal: destination path 'VQShape' already exists and is not an empty directory.
/content/VQShape


Instalamos algunas librerias necesarias para la cabeza de clasificacion

In [ ]:
!pip install lightning
!pip install sktime


### Cargamos el modelo VQShape preentrenado

Creamos la carpeta "checkpoints" y dentro de esa carpeta creamos una sub carpeta llamada "uea_dim256_codebook512" para cargar pesos .ckpt

In [ ]:
from vqshape.pretrain import LitVQShape
import torch

checkpoint = "checkpoints/uea_dim256_codebook512/VQShape.ckpt" # carga los pesos .ckpt
lit = LitVQShape.load_from_checkpoint(checkpoint, map_location="cuda") # Se mueve todo a GPU
base_model = lit.model
base_model.eval()

for p in base_model.parameters(): # Se congela (requires_grad = False) porque no  
    p.requires_grad = False      #vamos a entrenar VQShape, solo lo usamos como “feature extractor”


### Cargamos dataset Finger Movements (sktime)

Utilizamos sktime para cargar los archivos .ts oficiales del dataset Finger Movements.
Este dataset contiene señales multivariadas de EEG para dos clases:

- left
- right

Convertimos las etiquetas de texto a enteros (0 y 1) para entrenar el clasificador.

In [3]:
from sktime.datasets import load_from_tsfile
import numpy as np

X_train, y_train = load_from_tsfile("/content/FingerMovements_TRAIN.ts", return_data_type="numpy3D")
X_test, y_test   = load_from_tsfile("/content/FingerMovements_TEST.ts",  return_data_type="numpy3D")

label_map = {"left": 0, "right": 1}
y_train = np.array([label_map[y] for y in y_train])
y_test  = np.array([label_map[y] for y in y_test])


Imprimimos el modelo para tener una guia clara

In [ ]:
print(base_model)


### Preprocesamiento de señales y extracción de histogramas con VQShape

Antes de alimentar los datos al clasificador, es necesario preprocesar cada señal y obtener su representación discreta con VQShape. Para ello se definen dos funciones:

1. preprocess_signal() – Adaptación de la señal al modelo VQShape
Esta función:

- Convierte la muestra a tensor en GPU.
- Interpola cada canal a longitud 512 (requisito de VQShape).
- Reorganiza la matriz para tratar cada canal como una serie temporal independiente.

2. get_histogram() – Obtención del embedding discreto (histograma del codebook)
Aquí se usa el modelo VQShape en modo tokenize para:

- Obtener los códigos cuantizados de cada canal.
- Extraer el histograma de activaciones del codebook.
- Promediar los histogramas de los 28 canales para obtener un vector de 512 dimensiones.

In [ ]:
import torch.nn.functional as F
from einops import rearrange

def preprocess_signal(x):
    x = torch.tensor(x).float().cuda()       # (28,50)
    x = F.interpolate(x.unsqueeze(0), size=512, mode='linear').squeeze(0)  # (28,512)
    x = rearrange(x, 'c t -> (c) t')         # (28,512) → 28 univariates
    return x


def get_histogram(x):
    reps, _ = base_model(x, mode="tokenize")
    hist = reps["histogram"]   # (28, codebook_size)
    hist = hist.float().mean(dim=0)    # → vector (codebook_size,)
    return hist

### Cálculo masivo de histogramas para todo el dataset

Para acelerar el entrenamiento, se precomputan los histogramas de todas las muestras:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Pre-calcula TODOS los histogramas 
def get_features(X_data):
    base_model.eval()
    all_hists = []
    print(f"Calculando {len(X_data)} histogramas...")
    with torch.no_grad():
        for i in range(len(X_data)):
            x = preprocess_signal(X_data[i]) 
            hist = get_histogram(x)          
            all_hists.append(hist)
    return torch.stack(all_hists) # Forma: (N_samples, 512)

### Extracción de histogramas + definición del clasificador

Para cada serie temporal del dataset aplicamos:
- preprocess_signal: normalización y segmentación.
- get_histogram: codificación a tokens VQ y construcción de histogramas (512 bins).

El resultado (H_train, H_test) es un vector de longitud 512 por muestra:

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau # Importamos el scheduler
import numpy as np
import copy # Para guardar el mejor modelo


# Imprimimos los histogramas
print("--- Paso 1: Calculando Features ---")
H_train = get_features(X_train)
H_test  = get_features(X_test)
y_train_tensor = torch.tensor(y_train).long().cuda()
y_test_tensor  = torch.tensor(y_test).long().cuda()
print("Features calculados.")

### Modelo de clasificación

Implementamos un clasificador simple y lineal:
- Entrada: histograma de 512 dimensiones.
- Salida: dos clases (left / right).

Este clasificador se entrena desde cero, usando VQShape únicamente como extractor de tokens.

In [ ]:
# Definimos  el Clasificador
class SimpleClassifier(nn.Module):
    def __init__(self, hist_dim=512):
        super().__init__()
        self.net = nn.Linear(hist_dim, 2)

    def forward(self, h):
        return self.net(h)

clf = SimpleClassifier(hist_dim=512).cuda()

### Optimizador, scheduler y dataloaders

Se utiliza:

- Adam con lr=1e-3 y weight_decay=1e-5 (igual que el repo original).
- CrossEntropyLoss para clasificación binaria en logits.
- Scheduler ReduceLROnPlateau: reduce el LR si no mejora el accuracy en test, ayudando a estabilizar el entrenamiento.

In [ ]:
# Definimos el Optimizador y Scheduler
# El repo usa weight_decay=1e-5 y lr=1e-3
optimizer = torch.optim.Adam(clf.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

# Usamos un scheduler para reducir el LR si el test_acc deja de mejorar
# Esto ayuda a "afinar" el modelo y evitar sobreajuste
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10)

### Entrenamiento con early stopping y LR scheduler

Durante cada época se calculan:

- accuracy en entrenamiento
- accuracy en test

Mecanismos implementados:

- Scheduler: Reduce la tasa de aprendizaje al no mejorar el accuracy.
- Early Stopping: Si el modelo no mejora durante 30 épocas, se detiene el entrenamiento automáticamente.
- Guardado del mejor modelo: Se almacena en memoria (best_model_state) para cargarlo al final.

In [ ]:
# DataLoaders
BATCH_SIZE = 32
train_dataset = TensorDataset(H_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = TensorDataset(H_test, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Bucle de Entrenamiento con Early Stopping y Scheduler 
print("\n--- Paso 2: Iniciando entrenamiento optimizado ---")

# Variables para guardar el mejor modelo
best_test_acc = 0.0
best_epoch = 0
best_model_state = None # Guardaremos los "pesos" del mejor modelo
MAX_EPOCHS = 150

for epoch in range(MAX_EPOCHS):
    clf.train()
    train_correct = 0
    train_total = 0

    for h_batch, y_batch in train_loader:
        logits = clf(h_batch)
        loss = criterion(logits, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = logits.argmax(1)
        train_correct += (preds == y_batch).sum().item()
        train_total += y_batch.size(0)

    train_acc = train_correct / train_total

    # Bucle de Evaluación (Validación) 
    clf.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for h_batch, y_batch in test_loader:
            logits = clf(h_batch)
            preds = logits.argmax(1)
            test_correct += (preds == y_batch).sum().item()
            test_total += y_batch.size(0)

    test_acc = test_correct / test_total

    # Lógica de Early Stopping y Scheduler 

    # El scheduler se alimenta con el accuracy de test
    scheduler.step(test_acc)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch
        best_model_state = copy.deepcopy(clf.state_dict()) # Guarda una copia de los pesos

        if (epoch + 1) % 5 == 0 or epoch == 0:
             print(f"Epoch {epoch+1:03d} | Train Acc: {train_acc:.3f} | Test Acc: {test_acc:.3f} (¡Nuevo Mejor!)")
    else:
        if (epoch + 1) % 10 == 0:
             print(f"Epoch {epoch+1:03d} | Train Acc: {train_acc:.3f} | Test Acc: {test_acc:.3f}")

    # Parada temprana si no mejora en 30 épocas
    if (epoch - best_epoch) > 30:
        print("Parando temprano por falta de mejora.")
        break

print(f"\nEntrenamiento finalizado.")
print(f"Mejor Test Accuracy: {best_test_acc:.3f} (en época {best_epoch+1})")

# Carga los mejores pesos en el modelo
if best_model_state:
    clf.load_state_dict(best_model_state)

--- Paso 1: Calculando Features ---
Calculando 316 histogramas...
Calculando 100 histogramas...
Features calculados.

--- Paso 2: Iniciando entrenamiento optimizado ---
Epoch 001 | Train Acc: 0.506 | Test Acc: 0.510 (¡Nuevo Mejor!)
Epoch 010 | Train Acc: 0.630 | Test Acc: 0.510
Epoch 020 | Train Acc: 0.684 | Test Acc: 0.540
Epoch 030 | Train Acc: 0.680 | Test Acc: 0.520
Epoch 040 | Train Acc: 0.703 | Test Acc: 0.490
Parando temprano por falta de mejora.

Entrenamiento finalizado.
Mejor Test Accuracy: 0.590 (en época 9)


### Guardamos el mejor modelo

Al finalizar el entrenamiento guardamos los pesos del mejor modelo utilizando
PyTorch (.pt) para poder reutilizarlo en inferencia.

In [ ]:
# Guardar el mejor modelo 
torch.save(best_model_state, "best_eeg_classifier.pt")
print("Modelo guardado como best_eeg_classifier.pt")


Modelo guardado como best_eeg_classifier.pt


### Cargamos el modelo entrenado para inferencia

Se reconstruye el clasificador y se cargan los pesos entrenados.
El modelo se coloca en modo evaluación (eval()).

In [81]:
clf = SimpleClassifier(hist_dim=512).cuda()
clf.load_state_dict(torch.load("best_eeg_classifier.pt"))
clf.eval()


SimpleClassifier(
  (net): Linear(in_features=512, out_features=2, bias=True)
)

### Ejemplo de predicción en un índice del test

Tomamos un ejemplo del conjunto de test, lo procesamos igual que en entrenamiento y obtenemos la predicción del clasificador.

Se compara la clase predicha vs. la etiqueta real para validar el correcto funcionamiento.

In [85]:
i = 0   # índice

with torch.no_grad():
    x = preprocess_signal(X_test[i])
    hist = get_histogram(x)
    pred = clf(hist.unsqueeze(0)).argmax(1).item()

print("Predicción:", "right" if pred == 1 else "left")
print("Etiqueta real:", "right" if y_test[i] == 1 else "left")


Predicción: right
Etiqueta real: right
